Training and Test datasets  
이제부터 data와 test로 데이터셋을 반드시 나눔  

In [ ]:

# Lab 7 Learning rate and Evaluation
import tensorflow as tf
tf.set_random_seed(777)  # for reproducibility

x_data = [[1, 2, 1],
          [1, 3, 2],
          [1, 3, 4],
          [1, 5, 5],
          [1, 7, 5],
          [1, 2, 5],
          [1, 6, 6],
          [1, 7, 7]]
y_data = [[0, 0, 1],
          [0, 0, 1],
          [0, 0, 1],
          [0, 1, 0],
          [0, 1, 0],
          [0, 1, 0],
          [1, 0, 0],
          [1, 0, 0]]

# Evaluation our model using this test dataset
x_test = [[2, 1, 1],
          [3, 1, 2],
          [3, 3, 4]]
y_test = [[0, 0, 1],
          [0, 0, 1],
          [0, 0, 1]]

Multinomial Classification에 해당하는 모델이므로,   
softmax classification과 cross entropy cost함수를 사용

In [ ]:
X = tf.placeholder("float", [None, 3])
Y = tf.placeholder("float", [None, 3])

W = tf.Variable(tf.random_normal([3, 3]))
b = tf.Variable(tf.random_normal([3]))

# tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits), dim)
hypothesis = tf.nn.softmax(tf.matmul(X, W) + b)

# Cross entropy cost/loss
cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis), axis=1))
# Try to change learning_rate to small numbers
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

예측값은 원핫인코딩을 시켜 prediction에 저장하고  
prediction과 결과값이 같은지 일치하는지 확인(tf.equal() method)  

In [ ]:
# Correct prediction Test model
prediction = tf.argmax(hypothesis, 1)
is_correct = tf.equal(prediction, tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

학습과정:  
Training data만 가지고 학습을 시킨다. Optimizer를 실행할 때 feed해주는 데이터만 확인하면 된다.  
이후의 feed는 test데이터를 주고 평가하는 부분   


In [ ]:
# Launch graph
with tf.Session() as sess:
    # Initialize TensorFlow variables
    sess.run(tf.global_variables_initializer())

    for step in range(201):
        cost_val, W_val, _ = sess.run([cost, W, optimizer], feed_dict={X: x_data, Y: y_data})
        print(step, cost_val, W_val)

    # predict
    print("Prediction:", sess.run(prediction, feed_dict={X: x_test}))
    # Calculate the accuracy
    print("Accuracy: ", sess.run(accuracy, feed_dict={X: x_test, Y: y_test}))

Jupyter performance limit 해결:   
jupyter.textOutputLimit

---

Learning rate: NaN  
기존에는 0.1~0.01의 값만 주었음.  
1. 너무 크면, 오히려 minimum에서 멀어지는 overshooting(학습(수렴)되지 않고 발산되는)이 발생
2. 너무 작으면, 너무 느리게 학습되거나 local minimum에 갇혀버려서 학습이 되지 않을 수 있다.

확인하는 방법: 

In [17]:
X = tf.placeholder("float", [None, 3])
Y = tf.placeholder("float", [None, 3])

W = tf.Variable(tf.random_normal([3, 3]))
b = tf.Variable(tf.random_normal([3]))

# tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits), dim)
hypothesis = tf.nn.softmax(tf.matmul(X, W) + b)

# Cross entropy cost/loss
cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis), axis=1))
# Try to change learning_rate to small numbers
optimizer = tf.train.GradientDescentOptimizer(learning_rate=1.5).minimize(cost)
# Correct prediction Test model
prediction = tf.argmax(hypothesis, 1)
is_correct = tf.equal(prediction, tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

In [18]:
# Launch graph
with tf.Session() as sess:
    # Initialize TensorFlow variables
    sess.run(tf.global_variables_initializer())

    for step in range(201):
        cost_val, W_val, _ = sess.run([cost, W, optimizer], feed_dict={X: x_data, Y: y_data})
        print(step, cost_val, W_val)

    # predict
    print("Prediction:", sess.run(prediction, feed_dict={X: x_test}))
    # Calculate the accuracy
    print("Accuracy: ", sess.run(accuracy, feed_dict={X: x_test, Y: y_test}))

0 2.8577209 [[ 1.2888176  -1.8693092  -1.8167381 ]
 [ 1.6811122   1.8251038  -3.0029712 ]
 [ 0.24722767  1.7532312  -4.524243  ]]
1 12.50219 [[ 1.2803833 -2.423375  -1.2542381]
 [ 3.1213708 -1.1151547 -1.5029712]
 [ 1.9673705 -1.2794117 -3.2117429]]
2 26.371323 [[ 0.15538335 -1.8608749  -0.6917381 ]
 [-1.0036292   1.5098453  -0.00297117]
 [-2.1576295   1.5330883  -1.8992429 ]]
3 13.201395 [[ 0.5299828  -2.7957983  -0.13141418]
 [ 1.433069   -2.4224849   1.4926611 ]
 [ 0.27946925 -2.2143188  -0.58893466]]
4 14.219437 [[-0.5000918  -2.2332983   0.33616042]
 [-2.4387455   0.20251513  2.7394757 ]
 [-3.6659565   0.59818125  0.5439913 ]]
5 22.0958 [[-1.2509179e-01 -1.6708227e+00 -6.0131508e-01]
 [-1.2454987e-03  2.8274643e+00 -2.3229737e+00]
 [-1.2284565e+00  3.4106030e+00 -4.7059307e+00]]
6 nan [[nan nan nan]
 [nan nan nan]
 [nan nan nan]]
7 nan [[nan nan nan]
 [nan nan nan]
 [nan nan nan]]
8 nan [[nan nan nan]
 [nan nan nan]
 [nan nan nan]]
9 nan [[nan nan nan]
 [nan nan nan]
 [nan nan nan

결과값에 nan을 확인할 수 있다.(발산)

--- 
learning rate가 너무 작을 경우
cost function이 너무 적게 줄어드는 것을 볼 수 있다.

In [19]:
X = tf.placeholder("float", [None, 3])
Y = tf.placeholder("float", [None, 3])

W = tf.Variable(tf.random_normal([3, 3]))
b = tf.Variable(tf.random_normal([3]))

# tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits), dim)
hypothesis = tf.nn.softmax(tf.matmul(X, W) + b)

# Cross entropy cost/loss
cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis), axis=1))
# Try to change learning_rate to small numbers
optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-8).minimize(cost)
# Correct prediction Test model
prediction = tf.argmax(hypothesis, 1)
is_correct = tf.equal(prediction, tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

In [20]:
# Launch graph
with tf.Session() as sess:
    # Initialize TensorFlow variables
    sess.run(tf.global_variables_initializer())

    for step in range(201):
        cost_val, W_val, _ = sess.run([cost, W, optimizer], feed_dict={X: x_data, Y: y_data})
        print(step, cost_val, W_val)

    # predict
    print("Prediction:", sess.run(prediction, feed_dict={X: x_test}))
    # Calculate the accuracy
    print("Accuracy: ", sess.run(accuracy, feed_dict={X: x_test, Y: y_test}))

0 2.5147169 [[-0.5212399  -0.58198166 -0.2430529 ]
 [-0.56880695 -0.19104433 -0.2269075 ]
 [ 0.22639586  1.1175408   0.5743505 ]]
1 2.5147169 [[-0.5212399  -0.58198166 -0.2430529 ]
 [-0.56880695 -0.19104435 -0.2269075 ]
 [ 0.22639588  1.1175408   0.5743505 ]]
2 2.5147169 [[-0.5212399  -0.58198166 -0.2430529 ]
 [-0.56880695 -0.19104436 -0.2269075 ]
 [ 0.22639589  1.1175408   0.5743505 ]]
3 2.5147169 [[-0.5212399  -0.58198166 -0.2430529 ]
 [-0.56880695 -0.19104438 -0.2269075 ]
 [ 0.2263959   1.1175408   0.5743505 ]]
4 2.5147166 [[-0.5212399  -0.58198166 -0.2430529 ]
 [-0.56880695 -0.19104439 -0.2269075 ]
 [ 0.22639592  1.1175408   0.5743505 ]]
5 2.5147166 [[-0.5212399  -0.58198166 -0.2430529 ]
 [-0.56880695 -0.1910444  -0.2269075 ]
 [ 0.22639593  1.1175408   0.5743505 ]]
6 2.5147166 [[-0.5212399  -0.58198166 -0.2430529 ]
 [-0.56880695 -0.19104442 -0.2269075 ]
 [ 0.22639595  1.1175408   0.5743505 ]]
7 2.5147166 [[-0.5212399  -0.58198166 -0.2430529 ]
 [-0.56880695 -0.19104443 -0.2269075 ]


---
NaN이 발생하는 또 다른 경우:  
Non-normalized inputs  
이론적으로는 최저점을 향해 이동했으나 실제로는 그래프 밖으로 벗어난 경우.  

non-normalized: 매우 큰 값이 입력으로 주어짐

In [21]:
import tensorflow as tf
import numpy as np
tf.set_random_seed(777)  # for reproducibility


xy = np.array([[828.659973, 833.450012, 908100, 828.349976, 831.659973],
               [823.02002, 828.070007, 1828100, 821.655029, 828.070007],
               [819.929993, 824.400024, 1438100, 818.97998, 824.159973],
               [816, 820.958984, 1008100, 815.48999, 819.23999],
               [819.359985, 823, 1188100, 818.469971, 818.97998],
               [819, 823, 1198100, 816, 820.450012],
               [811.700012, 815.25, 1098100, 809.780029, 813.669983],
               [809.51001, 816.659973, 1398100, 804.539978, 809.559998]])


x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]

# placeholders for a tensor that will be always fed.
X = tf.placeholder(tf.float32, shape=[None, 4])
Y = tf.placeholder(tf.float32, shape=[None, 1])

W = tf.Variable(tf.random_normal([4, 1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

# Hypothesis
hypothesis = tf.matmul(X, W) + b

# Simplified cost/loss function
cost = tf.reduce_mean(tf.square(hypothesis - Y))

# Minimize
optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-5) #상당히 작은 Learning rate
train = optimizer.minimize(cost)


# Launch the graph in a session.
sess = tf.Session()
# Initializes global variables in the graph.
sess.run(tf.global_variables_initializer())

for step in range(101):
    cost_val, hy_val, _ = sess.run(
        [cost, hypothesis, train], feed_dict={X: x_data, Y: y_data})
    print(step, "Cost: ", cost_val, "\nPrediction:\n", hy_val)


0 Cost:  58554720000.0 
Prediction:
 [[170843.67]
 [345245.9 ]
 [271323.8 ]
 [189818.94]
 [223936.5 ]
 [225832.81]
 [206888.67]
 [263754.75]]
1 Cost:  6.4332716e+25 
Prediction:
 [[-5.6577952e+12]
 [-1.1389723e+13]
 [-8.9598837e+12]
 [-6.2808308e+12]
 [-7.4022949e+12]
 [-7.4645981e+12]
 [-6.8415626e+12]
 [-8.7106692e+12]]
2 Cost:  inf 
Prediction:
 [[1.8753527e+20]
 [3.7752775e+20]
 [2.9698745e+20]
 [2.0818663e+20]
 [2.4535906e+20]
 [2.4742421e+20]
 [2.2677285e+20]
 [2.8872692e+20]]
3 Cost:  inf 
Prediction:
 [[-6.2161094e+27]
 [-1.2513667e+28]
 [-9.8440498e+27]
 [-6.9006265e+27]
 [-8.1327574e+27]
 [-8.2012093e+27]
 [-7.5166923e+27]
 [-9.5702434e+27]]
4 Cost:  inf 
Prediction:
 [[2.0604133e+35]
 [4.1478233e+35]
 [3.2629429e+35]
 [2.2873058e+35]
 [2.6957121e+35]
 [2.7184012e+35]
 [2.4915088e+35]
 [3.1721861e+35]]
5 Cost:  inf 
Prediction:
 [[-inf]
 [-inf]
 [-inf]
 [-inf]
 [-inf]
 [-inf]
 [-inf]
 [-inf]]
6 Cost:  nan 
Prediction:
 [[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]


cost 값은 발산하고 있고, 결과값도 너무 커지는 결과를 볼 수 있다.

---
해결책: Normalized inputs(min-max scale)  
가장 큰 값을 1, 가장 작은 값을 0으로 하여 정규화시킴  


In [22]:
import tensorflow as tf
import numpy as np
tf.set_random_seed(777)  # for reproducibility


def min_max_scaler(data):
    numerator = data - np.min(data, 0)
    denominator = np.max(data, 0) - np.min(data, 0)
    # noise term prevents the zero division
    return numerator / (denominator + 1e-7)


xy = np.array(
    [
        [828.659973, 833.450012, 908100, 828.349976, 831.659973],
        [823.02002, 828.070007, 1828100, 821.655029, 828.070007],
        [819.929993, 824.400024, 1438100, 818.97998, 824.159973],
        [816, 820.958984, 1008100, 815.48999, 819.23999],
        [819.359985, 823, 1188100, 818.469971, 818.97998],
        [819, 823, 1198100, 816, 820.450012],
        [811.700012, 815.25, 1098100, 809.780029, 813.669983],
        [809.51001, 816.659973, 1398100, 804.539978, 809.559998],
    ]
)

# very important. It does not work without it.
xy = min_max_scaler(xy)
print(xy)

[[0.99999999 0.99999999 0.         1.         1.        ]
 [0.70548491 0.70439552 1.         0.71881782 0.83755791]
 [0.54412549 0.50274824 0.57608696 0.606468   0.6606331 ]
 [0.33890353 0.31368023 0.10869565 0.45989134 0.43800918]
 [0.51436    0.42582389 0.30434783 0.58504805 0.42624401]
 [0.49556179 0.42582389 0.31521739 0.48131134 0.49276137]
 [0.11436064 0.         0.20652174 0.22007776 0.18597238]
 [0.         0.07747099 0.5326087  0.         0.        ]]


In [23]:
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]

# placeholders for a tensor that will be always fed.
X = tf.placeholder(tf.float32, shape=[None, 4])
Y = tf.placeholder(tf.float32, shape=[None, 1])

W = tf.Variable(tf.random_normal([4, 1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

# Hypothesis
hypothesis = tf.matmul(X, W) + b

# Simplified cost/loss function
cost = tf.reduce_mean(tf.square(hypothesis - Y))

# Minimize
train = tf.train.GradientDescentOptimizer(learning_rate=1e-5).minimize(cost)

# Launch the graph in a session.
with tf.Session() as sess:
    # Initializes global variables in the graph.
    sess.run(tf.global_variables_initializer())

    for step in range(101):
        _, cost_val, hy_val = sess.run(
            [train, cost, hypothesis], feed_dict={X: x_data, Y: y_data}
        )
        print(step, "Cost: ", cost_val, "\nPrediction:\n", hy_val)

0 Cost:  2.5667095 
Prediction:
 [[1.713187 ]
 [2.9957201]
 [2.3465576]
 [1.5928736]
 [1.9746393]
 [2.0049214]
 [1.6920049]
 [2.0588913]]
1 Cost:  2.5665417 
Prediction:
 [[1.7131163]
 [2.995647 ]
 [2.3464963]
 [1.5928262]
 [1.9745836]
 [2.0048676]
 [1.6919664]
 [2.0588522]]
2 Cost:  2.566373 
Prediction:
 [[1.7130456]
 [2.9955738]
 [2.3464353]
 [1.5927789]
 [1.9745281]
 [2.0048134]
 [1.6919279]
 [2.0588126]]
3 Cost:  2.5662053 
Prediction:
 [[1.7129749]
 [2.9955006]
 [2.3463743]
 [1.5927316]
 [1.9744725]
 [2.0047596]
 [1.6918895]
 [2.0587735]]
4 Cost:  2.566037 
Prediction:
 [[1.7129042]
 [2.9954271]
 [2.3463132]
 [1.5926843]
 [1.974417 ]
 [2.0047054]
 [1.691851 ]
 [2.0587342]]
5 Cost:  2.565869 
Prediction:
 [[1.7128334]
 [2.9953542]
 [2.3462522]
 [1.592637 ]
 [1.9743614]
 [2.0046515]
 [1.6918125]
 [2.0586948]]
6 Cost:  2.565701 
Prediction:
 [[1.7127628]
 [2.9952807]
 [2.3461912]
 [1.5925896]
 [1.9743059]
 [2.0045977]
 [1.6917741]
 [2.0586555]]
7 Cost:  2.5655327 
Prediction:
 [[1.7